In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [2]:
 # Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import avg
from pyspark.sql.functions import format_number
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show(3)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
only showing top 3 rows



In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# https://kontext.tech/article/822/spark-sql-average-avg-calculation
# https://database.guide/4-functions-to-format-a-number-to-2-decimal-places-in-sql-server/

In [7]:
fourbed_query = """(SELECT YEAR(date) AS YEAR,
round(avg(price), 2),
count(date)
FROM home_sales WHERE bedrooms=4
GROUP BY YEAR
ORDER BY YEAR ASC)"""

print("Average 4-bedroom price by year and their quantity")
spark.sql(fourbed_query).show()

Average 4-bedroom price by year and their quantity
+----+--------------------+-----------+
|YEAR|round(avg(price), 2)|count(date)|
+----+--------------------+-----------+
|2019|            300263.7|       3120|
|2020|           298353.78|       3155|
|2021|           301819.44|       3169|
|2022|           296363.88|       1394|
+----+--------------------+-----------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# spark.sql("SELECT avg(price) FROM home_sales WHERE bedrooms==3 AND bathrooms==3").show()

threebed_query = """(SELECT YEAR(date) AS YEAR,
round(avg(price), 2),
count(date)
FROM home_sales WHERE bedrooms=3
AND bathrooms=3
GROUP BY YEAR
ORDER BY YEAR ASC)"""

print("Average 3-bed, 3-bath price by year and their quantity")
spark.sql(threebed_query).show()


Average 3-bed, 3-bath price by year and their quantity
+----+--------------------+-----------+
|YEAR|round(avg(price), 2)|count(date)|
+----+--------------------+-----------+
|2019|           287287.82|       1568|
|2020|           294204.16|       1553|
|2021|           294211.46|       1571|
|2022|           292725.69|        704|
+----+--------------------+-----------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# spark.sql("SELECT avg(price) FROM home_sales WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living >= 2000").show()

bigthreebed_query = """(SELECT YEAR(date) AS YEAR,
round(avg(price), 2),
count(date)
FROM home_sales WHERE bedrooms=3
AND bathrooms=3
AND floors=2
AND sqft_living >= 2000
GROUP BY YEAR
ORDER BY YEAR ASC)"""

print("Average price for a large 3-bed, 3-bath house by year and their quantity")
spark.sql(bigthreebed_query).show()


Average price for a large 3-bed, 3-bath house by year and their quantity
+----+--------------------+-----------+
|YEAR|round(avg(price), 2)|count(date)|
+----+--------------------+-----------+
|2019|           289859.14|        406|
|2020|           292289.09|        391|
|2021|           296330.96|        413|
|2022|           290242.99|        166|
+----+--------------------+-----------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT round(avg(view), 2), COUNT(view) FROM home_sales WHERE price >= 350000").show()

# print("--- %s seconds ---" % (time.time() - start_time))
print("--- %.2f seconds to complete query ---" % (time.time() - start_time))

+-------------------+-----------+
|round(avg(view), 2)|count(view)|
+-------------------+-----------+
|              32.26|      11125|
+-------------------+-----------+

--- 0.95 seconds to complete query ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT round(avg(view), 2), COUNT(view) FROM home_sales WHERE price >= 350000").show()

print("--- %.2f seconds to complete query ---" % (time.time() - start_time))


+-------------------+-----------+
|round(avg(view), 2)|count(view)|
+-------------------+-----------+
|              32.26|      11125|
+-------------------+-----------+

--- 0.74 seconds to complete query ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.parquet('parquet_home_sales_basic', mode='overwrite')

In [15]:
# 11. Read the parquet formatted data.
p_home_sales_df=spark.read.parquet('parquet_home_sales_basic')


In [16]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_home_sales')


In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT round(avg(view), 2), COUNT(view) FROM p_home_sales WHERE price >= 350000").show()

print("--- %.2f seconds to complete query ---" % (time.time() - start_time))

+-------------------+-----------+
|round(avg(view), 2)|count(view)|
+-------------------+-----------+
|              32.26|      11125|
+-------------------+-----------+

--- 0.60 seconds to complete query ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is still cached")
else:
  print("all clear")


all clear
